In [1]:
# -*- encoding: utf-8 -*-
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [2]:
import numpy as np
import logging

In [3]:
# 获取训练数据
def getTrainSet(inFile):
    # 文章标题集
    title_set = []
    # 训练集
    train_set=[]
    # 读入训练数据
    f=open(inFile)
    lines=f.readlines()
    for line in lines:
        article = line.replace('\n','').split('\t')
        title = article[0]
        title_set.append(title)
        content = article[1:]
        train_set.append(content)
    f.close()
    return (title_set,train_set)

In [4]:
from sklearn.preprocessing import LabelEncoder
# 处理标题数据里包含的分类，生成target标签
def processLable(text):
    target = []
    le = LabelEncoder()
    for t in text:
        catlg = t.split("#")[0]
        target.append(catlg)
    return le.fit_transform(target)

In [5]:
# 把文本内容转化为LSI向量
from gensim import corpora,models,utils,matutils
def vecTransformLSI(train_set):
    # 生成字典
    dictionary = corpora.Dictionary(train_set)
    dictionary.filter_extremes(no_below=1,no_above=1,keep_n=None)
    
    # 生成语料
    corpus = [dictionary.doc2bow(text) for text in train_set]
    
    #使用数字语料生成TFIDF模型
    tfidfModel = models.TfidfModel(corpus)

    #把全部语料向量化成TFIDF模式,这个tfidfModel可以传入二维数组
    tfidfVectors = tfidfModel[corpus]
    
    #通过TFIDF向量生成LSI模型,id2word表示编号的对应词典,num_topics表示主题数
    lsi = models.LsiModel(tfidfVectors, id2word=dictionary, num_topics=400)

    #把所有TFIDF向量变成LSI的向量
    corpus_vec = lsi[tfidfVectors]
    
    # 把预料库转化为scipy稀疏矩阵
    matrix = matutils.corpus2csc(corpus_vec).toarray()
    print "matrix shape=",matrix.shape
    return matrix

In [6]:
# 把文本内容转化为LDA向量
from gensim import corpora,models,utils,matutils
def vecTransformLDA(train_set):
    # 生成字典
    dictionary = corpora.Dictionary(train_set)
    dictionary.filter_extremes(no_below=1,no_above=1,keep_n=None)
    
    # 生成语料
    corpus = [dictionary.doc2bow(text) for text in train_set]
    
    #使用数字语料生成TFIDF模型
    tfidfModel = models.TfidfModel(corpus)

    #把全部语料向量化成TFIDF模式,这个tfidfModel可以传入二维数组
    tfidfVectors = tfidfModel[corpus]
    
    #通过TFIDF向量生成LDA模型,id2word表示编号的对应词典,num_topics表示主题数
    lda = models.LdaModel(tfidfVectors, id2word=dictionary, num_topics=400)

    #把所有TFIDF向量变成LDA的向量
    corpus_vec = lda[tfidfVectors]
    
    # 把预料库转化为scipy稀疏矩阵
    matrix = matutils.corpus2csc(corpus_vec).toarray()
    print "matrix shape=",matrix.shape
    return matrix

In [7]:
def trainModel():
    
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    
    inFile = "./data/all.txt"
    # 读入数据文件
    label,text = getTrainSet(inFile)
    # 获取标签，并写入文件
    target = processLable(label)
    np.save("./data/target.npy",target)
    
    # 转换数据，并写入文件
    vec = vecTransformLSI(text)
    np.save("./data/textData_LSI.npy",vec)
    
    # 转换数据，并写入文件
    vec = vecTransformLDA(text)
    np.save("./data/textData_LDA.npy",vec)

In [8]:
def main():
    
    trainModel()
    

In [9]:
if __name__ == '__main__':
    main()